In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext
import json

sc = SparkContext.getOrCreate()

review_path = r'C:\Users\11921\Downloads\review.json'
business_path = r'C:\Users\11921\Downloads\business.json'

reviewRDD = sc.textFile(review_path)
businessRDD = sc.textFile(business_path)

def keepIdAndState(eachBus):
    busJson = json.loads(eachBus)
    return (busJson['business_id'], busJson['state'])

NVBusId = businessRDD.map(lambda eachBus:keepIdAndState(eachBus))\
                         .filter(lambda idStateTuple:idStateTuple[1]=='NV')\
                         .map(lambda NVBus: NVBus[0]).collect()                

def keepUserAndBus(eachReview):
    reviewJson = json.loads(eachReview)
    return (reviewJson['user_id'], reviewJson['business_id'])
    
resultList = reviewRDD.map(lambda eachReview: keepUserAndBus(eachReview)).filter(lambda pair: pair[1] in NVBusId).collect()

outputFile = open('user_business_file.csv', 'w')
outputFile.write('user_id,business_id')
for pair in resultList:
    outputFile.write('\n'+pair[0]+','+pair[1])
outputFile.close()